In [1]:
import pandas as pd
import pathlib

In [2]:
curr_dir = pathlib.Path().resolve()
print(curr_dir)

/home/inductive-anks/yield-prediction/Yield-Prediction/Code Files/Tehsil-Village-Mapping


In [3]:
home_dir = curr_dir.parent.parent

In [2]:
village_list_df = pd.read_csv(home_dir.as_posix() + '/data/Final Data/Final-Merged-Data(2008-2011 to 2017-2020).csv')

### Number of Tehsils greater than one Village

In [3]:
grouped = village_list_df.groupby(['State Name', 'Year'])['Tehsil Name'].nunique().reset_index()

In [4]:
total_unique_tehsils_per_year = grouped.groupby('Year')['Tehsil Name'].sum().reset_index()
total_unique_tehsils_per_year.columns = ['Year', 'Total Number of unique Tehsils']

In [5]:
multi_village_tehsils = village_list_df.groupby(['Tehsil Name', 'Year'])['Village Name'].nunique().reset_index()
multi_village_tehsils = multi_village_tehsils[multi_village_tehsils['Village Name'] > 1]
multi_village_tehsils_per_year = multi_village_tehsils.groupby('Year')['Tehsil Name'].nunique().reset_index()
multi_village_tehsils_per_year.columns = ['Year', 'Total Number of unique Tehsils > 1 village']

In [6]:
final_tehsil_data = pd.merge(total_unique_tehsils_per_year, multi_village_tehsils_per_year, on='Year', how='left').fillna(0)

In [7]:
final_tehsil_data.head()

,Year,Total Number of unique Tehsils,Total Number of unique Tehsils > 1 village
0,2008-2011,746,154
1,2011-2014,695,88
2,2014-2017,793,173
3,2017-2020,787,154


In [8]:
final_tehsil_data_multiple_village = village_list_df[village_list_df['Tehsil Name'].isin(multi_village_tehsils['Tehsil Name'])]

In [9]:
final_tehsil_data_multiple_village.shape

(1768, 7)

In [10]:
final_tehsil_data_multiple_village.head()

,Zone No,Tehsil Code,Tehsil Name,District Name,Village Name,State Name,Year
43,I,1,Dhemaji,Dhemaji,na-pam,Assam,2008-2011
44,I,1,Dhemaji,Dhemaji,dushuti,Assam,2008-2011
45,I,2,Laluk,North Lakhimpur,bongalmara,Assam,2008-2011
46,I,2,Laluk,North Lakhimpur,miri gaon,Assam,2008-2011
47,I,2,Laluk,North Lakhimpur,anojuli,Assam,2008-2011


In [11]:
final_tehsil_data_multiple_village.to_csv(home_dir.as_posix() + '/data/Tehsil Data/Tehsil_Data_With_Multiple_Village.csv', index=False)

### Unique Tehsils with only 1 Village

In [12]:
grouped = village_list_df.groupby(['State Name', 'Year'])['Tehsil Name'].nunique().reset_index()

In [13]:
total_unique_tehsils_per_year = grouped.groupby('Year')['Tehsil Name'].sum().reset_index()

In [14]:
total_unique_tehsils_per_year

,Year,Tehsil Name
0,2008-2011,746
1,2011-2014,695
2,2014-2017,793
3,2017-2020,787


In [16]:
single_village_tehsils = village_list_df.groupby(['Tehsil Name', 'Year'])['Village Name'].nunique().reset_index()

In [17]:
single_village_tehsils.head()

,Tehsil Name,Year,Village Name
0,CHIDAMBARAM,2011-2014,1
1,(Dhula),2008-2011,2
2,1.0,2008-2011,1
3,10.0,2008-2011,1
4,11.0,2008-2011,1


In [18]:
single_village_tehsils = single_village_tehsils[single_village_tehsils['Village Name'] == 1]

In [19]:
single_village_tehsils.head()

,Tehsil Name,Year,Village Name
0,CHIDAMBARAM,2011-2014,1
2,1.0,2008-2011,1
3,10.0,2008-2011,1
4,11.0,2008-2011,1
5,12.0,2008-2011,1


In [21]:
final_tehsil_data_single_village = village_list_df[village_list_df['Tehsil Name'].isin(single_village_tehsils['Tehsil Name'])]

In [22]:
final_tehsil_data_single_village.shape 

(2621, 7)

In [ ]:
final_tehsil_data_single_village.to_csv('Tehsil_Data_With_Single_Village.csv', index=False)